# Dependencies

In [1]:
import kaggle

# Download and load database

## Extract data from Kaggle ID dataset

For this case: https://www.kaggle.com/datasets/deepshah16/song-lyrics-dataset

In [2]:
kaggle_dataset_id = "deepshah16/song-lyrics-dataset"

In [3]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files(kaggle_dataset_id, path='./', unzip=True)

## Concatenate 2 dataframes

In [6]:
import pandas as pd

In [7]:
df_bts = pd.read_csv("./csv/BTS.csv")
df_lady_gaga = pd.read_csv("./csv/LadyGaga.csv")

In [8]:
df_lyrics = pd.concat([df_bts, df_lady_gaga], axis=0)

In [13]:
df_lyrics.to_csv('lyrics.csv', index=False)

## Save to duckdb

```
db_path = "data.duckdb"
table_name = "song_lyrics"
save_to_duckdb(df, table_name, db_path)
```

# Visualize database

In [4]:
%load_ext sql
%sql duckdb://

The 'toml' package isn't installed. To load settings from the pyproject.toml file, install with: pip install toml

Connecting to 'duckdb://'

In [14]:
%%sql
SELECT * FROM lyrics.csv LIMIT 5

Running query in 'duckdb://'

Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,BTS (방탄소년단),Dynamite,BE,2020.0,2020-08-21,jungkook 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight jungkook shoes on get up in the morn' cup of milk let's rock and roll king kong kick the drum rolling on like a rolling stone sing song when i'm walkin' home jump up to the top lebron dingdong call me on my phone ice tea and a game of ping pong pre rm jhope this is gettin' heavy can you hear the bass boom i'm ready woohoo life is sweet as honey yeah this beat chaching like money huh disco overload i'm into that i'm good to go i'm diamond you know i glow up hey so let's go jungkook jimin 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight hey shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh v rm bring a friend join the crowd whoever wanna come along word up talk the talk just move like we off the wall day or night the sky's alight so we dance to the break of dawn hey ladies and gentlemen i got the medicine so you should keep ya eyes on the ball pre suga jimin rm huh this is gettin' heavy can you hear the bass boom i'm ready woohoo life is sweet as honey yeah this beat chaching like money huh disco overload i'm into that i'm good to go i'm diamond you know i glow up let's go jungkook v 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight hey shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh post jungkook jimin jin dynnanana nanananana nanana life is dynamite dynnanana nanananana nanana life is dynamite shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh jungkook jhope jimin v dynnanana nana nana ayy dynnanana nana nana ayy dynnanana nana nana ayy light it up like dynamite dynnanana nana nana ayy dynnanana nana nana ayy dynnanana nana nana ayy light it up like dynamite jimin jungkook jin 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight shinin' through the city with a little funk and soul so i'ma light it up like dynamite this is ah 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight alight oh shinin' through the city with a little funk and soul so i'ma light light it it up up like like dynamite dynamite woahohoh post jungkook jimin v dynnanana nanananana nanana life is dynamite life is dynamite dynnanana nanananana nanana life is dynamite oh shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh
1,BTS (방탄소년단),FAKE LOVE,LOVE YOURSELF 轉 ‘Tear’,2018.0,2018-05-18,방탄소년단의 fake love 가사 v jungkook 널 위해서라면 난 슬퍼도 기쁜 척 할 수가 있었어 널 위해서라면 난 아파도 강한 척 할 수가 있었어 사랑이 사랑만으로 완벽하길 내 모든 약점들은 다 숨겨지길 이뤄지지 않는 꿈속에서 피울 수 없는 꽃을 키웠어 hook jimin jin i'm so sick of this fake love fake love fake love i'm so sorry but it's fake love fake love fake love rm rm jungkook jhope i wanna be a good man just for you 세상을 줬네 just for you 전부 바꿨어 just for you now i don't know me who are you 우리만의 숲 너는 없었어 내가 왔던 route 잊어버렸어 나도 내가 누구였는지도 잘 모르게 됐어 거울에다 지껄여봐 너는 대체 누구니 pre jimin jin 널 위해서라면 난 슬퍼도 기쁜 척 할 수가 있었어 널 위해서라면 난 아파도 강한 척 할 수가 있었어 사랑이 사랑만으로 완벽하길 내 모든 약점들은 다 숨겨지길 이뤄지지 않는 꿈속에서 피울 수 없는 꽃을 키웠어 jungkook v love you so bad love you so bad 널 위해 예쁜 거짓을 빚어내 love it's so mad love it's so mad 날 지워 너의 인형이 되려 해 love you so bad love you so bad 널 위해 예쁜 거짓을 빚어내 love it's so mad love it's so mad 날 지워 너의 인형이 되려 해 hook jimin jin i'm so sick of this fake love fake love fake love i'm so sorry but it's fake love fake love fake love rm suga why you sad i don't know 난 몰라 웃어봐 사랑해 말해봐 나를 봐 나조차도 버린 나 너조차 이해할 수 없는 나 낯설다 하네 니가 좋아하던 나로 변한 내가 아니라 하네 예전에 니가 잘 알고 있던 내가 아니긴 뭐가 아냐 난 눈 멀었어 사랑은 뭐가 사랑 it's all fake love jimin jin woo i don't know i don't know i don't know why woo 나도 날 나도 날 모르겠어 woo i just know i just know i just know why 'cause it's all fake love fake love fake love jungkook jimin love you so bad

# Insights

### Count the number of songs per artist

In [16]:
query = f"""SELECT Artist, COUNT(*) FROM lyrics.csv GROUP BY Artist;"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}


Running query in 'duckdb://'

Artist,count_star()
BTS (방탄소년단),278
Lady Gaga,402


### Get the songs released in a specific year

In [17]:
query = f"""SELECT * FROM lyrics.csv WHERE Year = '2020';"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,BTS (방탄소년단),Dynamite,BE,2020.0,2020-08-21,jungkook 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight jungkook shoes on get up in the morn' cup of milk let's rock and roll king kong kick the drum rolling on like a rolling stone sing song when i'm walkin' home jump up to the top lebron dingdong call me on my phone ice tea and a game of ping pong pre rm jhope this is gettin' heavy can you hear the bass boom i'm ready woohoo life is sweet as honey yeah this beat chaching like money huh disco overload i'm into that i'm good to go i'm diamond you know i glow up hey so let's go jungkook jimin 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight hey shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh v rm bring a friend join the crowd whoever wanna come along word up talk the talk just move like we off the wall day or night the sky's alight so we dance to the break of dawn hey ladies and gentlemen i got the medicine so you should keep ya eyes on the ball pre suga jimin rm huh this is gettin' heavy can you hear the bass boom i'm ready woohoo life is sweet as honey yeah this beat chaching like money huh disco overload i'm into that i'm good to go i'm diamond you know i glow up let's go jungkook v 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight hey shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh post jungkook jimin jin dynnanana nanananana nanana life is dynamite dynnanana nanananana nanana life is dynamite shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh jungkook jhope jimin v dynnanana nana nana ayy dynnanana nana nana ayy dynnanana nana nana ayy light it up like dynamite dynnanana nana nana ayy dynnanana nana nana ayy dynnanana nana nana ayy light it up like dynamite jimin jungkook jin 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight shinin' through the city with a little funk and soul so i'ma light it up like dynamite this is ah 'cause i i i'm in the stars tonight so watch me bring the fire and set the night alight alight oh shinin' through the city with a little funk and soul so i'ma light light it it up up like like dynamite dynamite woahohoh post jungkook jimin v dynnanana nanananana nanana life is dynamite life is dynamite dynnanana nanananana nanana life is dynamite oh shinin' through the city with a little funk and soul so i'ma light it up like dynamite woahohoh
18,BTS (방탄소년단),ON,MAP OF THE SOUL: 7,2020.0,2020-02-21,방탄소년단 on 가사 jimin v i can't understand what people are sayin' 어느 장단에 맞춰야 될지 한 발자국 떼면 한 발자국 커지는 shadow 잠에서 눈을 뜬 여긴 또 어디 어쩜 서울 또 new york or paris 일어나니 휘청이는 몸 rm look look at my feet look down 날 닮은 그림자 흔들리는 건 이놈인가 아니면 내 작은 발끝인가 두렵잖을 리 없잖아 다 괜찮을 리 없잖아 그래도 i know 서툴게 i flow 저 까만 바람과 함께 날아 pre jungkook jimin jin hey nanana 미치지 않으려면 미쳐야 해 hey nanana 나를 다 던져 이 두 쪽 세상에 hey nanana cant hold me down 'cause you know im a fighter 제 발로 들어온 아름다운 감옥 find me and i'm gonna live with ya jungkook jimin ayyoh 가져와 bring the pain oh yeah ayyoh 올라타봐 bring the pain oh yeah rain be pourin' sky keep fallin' everyday ohnanana ayyoh 가져와 bring the pain oh yeah jhope bring the pain 모두 내 피와 살이 되겠지 bring the pain no fear 방법을 알겠으니 작은 것에 breathe 그건 어둠 속 내 산소와 빛 내가 나에게 하는 것들의 힘 넘어져도 다시 일어나 scream 4 suga 넘어져도 다시 일어나 scream 언제나 우린 그랬으니 설령 내 무릎이 땅에 닿을지언정 파묻히지 않는 이상 그저 그런 해프닝쯤 될 거란 걸 win no matter what win no matter what win no matter what 네가 뭐라던 누가 뭐라던 i don't give a uh i don't give a uh i don't give a uh pre v jimin hey nanana 미치지 않으려면 미쳐야 해 hey nanana 나를 다 던져 이 두 쪽 세상에 hey nanana cant hold me down 'cause you know im a fighter 제 발로 들어온 아름다운 감옥 find me and i'm gonna live with ya v jin ayyoh 가져와 bring the pain oh yeah ayyoh 올라타봐 bring the pain oh yeah rain be pourin' sky keep fallin' everyday ohnanana ayyoh 가져와 bring the 

### Find the most common words in the lyrics

In [18]:
query = f"""SELECT Lyric, COUNT(*) as count FROM lyrics.csv GROUP BY Lyric ORDER BY count DESC LIMIT 10;"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Count the number of songs per album

In [19]:
query = f"""SELECT Album, COUNT(*) FROM lyrics.csv GROUP BY Album;"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

Album,count_star()
BE,7
LOVE YOURSELF 轉 ‘Tear’,13
None,114
You Never Walk Alone,7
Love Yourself 承 ‘Her’,12
NOW That’s What I Call Music! 65 [US],1
LOVE YOURSELF 結 ‘Answer’,18
WINGS,15
Face Yourself,12
MAP OF THE SOUL: PERSONA,7


### Get the songs by a specific artist

In [21]:
query = f"""SELECT * FROM lyrics.csv WHERE Artist = 'Lady Gaga';"""
# Use JupySQL magic %sql to execute the query
%sql {{query}}

Running query in 'duckdb://'

Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,Lady Gaga,Do What U Want,ARTPOP,2013.0,2013-10-21,lady gaga r kelly yeah oh turn the mic up yeah yeah eh eh eh eh eh eh eh oh eh eh eh eh eh eh eh oh lady gaga i i feel good i walk alone but then i trip upon myself and i fall i i stand up and then i'm okay but then you print some shit that makes me wanna scream pre lady gaga so do what you want what you want with my body do what you want don't stop let's party do what you want what you want with my body do what you want what you want with my body write what you want say what you want 'bout me if you're wondering know that i'm not sorry do what you want what you want with my body what you want with my body lady gaga you can't have my heart and you won't use my mind but do what you want with my body do what you want with my body you cant stop my voice 'cause you don't own my life but do what you want with my body do what you want with my body r kelly early morning longer nights yeah tom ford private flights yeah crazy schedule fast life i wouldn't trade it in 'cause it's our life but let's slow it down i could be the drink in your cup i could be the green in your blunt your pusher man yeah i got what you want you wanna escape oh all of the crazy shit let go you're the marilyn i'm the president and i love to hear you sing girl pre r kelly do what i want do what i want with your body do what i want do what i want with your body back of the club taking shots gettin' naughty no invitations it's a private party do what i want do what i want with your body do what i want do what i want with your body yeah we taking these haters and we roughin' 'em up and we layin the cut like we don't give a fuck lady gaga r kelly you can't have my heart and you won't use my mind my mind but do what you want with my body yeah do what you want with my body with your body you cant stop my voice 'cause you don't own my life but do what you want with my body do what you want with my body body lady gaga sometimes i'm scared i suppose if you ever let me go i would fall apart if you break my heart so just take my body and don't stop the party lady gaga r kelly both you can't have my heart and help me now you won't use my mind but do what you want with my body do what you want with my body with your body you cant stop my voice 'cause you don't own my life you but do what you want with my body what i want when i want when i want do what you want with my body lady gaga do what you want with me what you want with my body do what you want with me what you want with my body world do what you want with me what you want with my body do what you want with me what you want with my body world help me now what you want with my body do what you want with my body do what you want with my body want you want with my body world
1,Lady Gaga,Always Remember Us This Way,A Star is Born (Original Motion Picture Soundtrack),2018.0,2018-10-05,that arizona sky burnin' in your eyes you look at me and babe i wanna catch on fire its buried in my soul like california gold you found the light in me that i couldnt find so when i'm all choked up and i can't find the words every time we say goodbye baby it hurts when the sun goes down and the band won't play i'll always remember us this way lovers in the night poets tryna write we don't know how to rhyme but damn we try but all i really know you're where i wanna go the part of me that's you will never die so when i'm all choked up and i can't find the words every time we say goodbye baby it hurts when the sun goes down and the band won't play i'll always remember us this way oh yeah i don't wanna be just a memory baby yeah hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hoo hooooooooo so when i'm all choked up and i can't find the words every time we say goodbye baby it hurts when the sun goes down and the band won't play i'll always remember us this way way yeah when you look at me and the whole world fades i'll always rem